In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import json
import pandas

In [2]:
directory = '/Users/maciejglazek/Desktop/UNI/Python/rap_lyrics/lyrics/'

'''for filename in os.listdir(directory):
    print(filename)'''
    
lyrics = []
for filename in os.listdir(directory):
    try:   
        with open(directory + filename) as f:
            try:
                data = json.load(f)
                lyrics.append(data["Lyrics"])
            except: continue
    except: continue
        

In [3]:
len(lyrics)

1195

In [4]:
flat_lyrics = []
for song in lyrics:
    for verse in song:
        for word in verse:
            for letter in word:
                flat_lyrics.append(letter)

In [5]:
len(flat_lyrics)

4110005

In [6]:
# The unique characters in the file
good = ['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [7]:
flat_lyrics_clean = []
for letter in flat_lyrics: 
    if letter in good: flat_lyrics_clean.append(letter)
    else: continue

In [8]:
len(flat_lyrics_clean)

4102521

In [9]:
vocab = sorted(set(flat_lyrics_clean))
print ('{} unique characters'.format(len(vocab)))
print(vocab)

87 unique characters
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [10]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in flat_lyrics_clean])

In [11]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  '*' :  11,
  '+' :  12,
  ',' :  13,
  '-' :  14,
  '.' :  15,
  '/' :  16,
  '0' :  17,
  '1' :  18,
  '2' :  19,
  ...
}


In [12]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(flat_lyrics_clean[:13]), text_as_int[:13]))

['[', 'V', 'e', 'r', 's', 'e', ' ', '1', ':', ' ', 'I', 's', 'a'] ---- characters mapped to int ---- > [59 54 65 78 79 65  1 18 27  1 41 79 61]


In [13]:
# The maximum length sentence we want for a single input in characters
seq_length = 350
examples_per_epoch = len(flat_lyrics_clean)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

[
V
e
r
s


In [14]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"[Verse 1: Isaiah Rashad]\nI take a drag at the square, I feel anxious, spit dangerous\nAs the verbal appears, it's reflectin' my perspective\nBeer goggle and fear, role model so hollow\nShadow adolescence and a gaggle of them bitches\nRoad to the rich is still paved with the ditches\nGet caught up in the hype, your career is for a night\nI ain't these othe"
"r niggas, 'cause these niggas is fake\nThey ain't poppin' shot, these niggas is Papa Doc\nFine as the shine, beam me up, and I'm ridin' Spock\nNo cryin' 'til I'm in a pine box\nI got the bitches hittin' Mikey like where Zay at?\nProlly fuckin' on her friend, prolly L.A. at\nProlly plottin' on a mil', stayin' well fed\nBitch, don't run your mouth, we don't "
"play that\nMomma taught me better, can't count on niggas\nCan't count on weather  go figure, nigga\nThey ain't worried 'bout this cheddar\nThem niggas just sky divin' in my propellers, I don't feel 'em\nAin't tryna be no freshman, I'm chillin'\n'Cause they'll bring a knife to 

In [15]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [16]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "[Verse 1: Isaiah Rashad]\nI take a drag at the square, I feel anxious, spit dangerous\nAs the verbal appears, it's reflectin' my perspective\nBeer goggle and fear, role model so hollow\nShadow adolescence and a gaggle of them bitches\nRoad to the rich is still paved with the ditches\nGet caught up in the hype, your career is for a night\nI ain't these oth"
Target data: "Verse 1: Isaiah Rashad]\nI take a drag at the square, I feel anxious, spit dangerous\nAs the verbal appears, it's reflectin' my perspective\nBeer goggle and fear, role model so hollow\nShadow adolescence and a gaggle of them bitches\nRoad to the rich is still paved with the ditches\nGet caught up in the hype, your career is for a night\nI ain't these othe"


In [17]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 59 ('[')
  expected output: 54 ('V')
Step    1
  input: 54 ('V')
  expected output: 65 ('e')
Step    2
  input: 65 ('e')
  expected output: 78 ('r')
Step    3
  input: 78 ('r')
  expected output: 79 ('s')
Step    4
  input: 79 ('s')
  expected output: 65 ('e')


In [18]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 350), (64, 350)), types: (tf.int64, tf.int64)>

In [19]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [20]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [21]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 350, 87) # (batch_size, sequence_length, vocab_size)


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           22272     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 87)            89175     
Total params: 4,049,751
Trainable params: 4,049,751
Non-trainable params: 0
_________________________________________________________________


In [24]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [25]:
sampled_indices

array([26, 37,  9, 33, 59, 45, 47, 25, 34, 36, 78, 66, 40, 82, 62, 56, 68,
       40, 39, 29, 14, 40, 80, 63, 62, 64, 81, 68, 77, 75, 84, 86, 45, 39,
       18, 65, 84, 74, 58, 25, 84,  0, 39, 51,  6, 65, 45,  3, 11, 29, 84,
        0, 63, 69, 74, 24, 24, 85, 20, 31, 64, 85, 71, 15, 84, 75, 50, 62,
       55,  0, 61, 73, 77,  8, 60, 20, 51, 23, 22, 35, 55, 81, 52,  1, 76,
       41, 58, 51, 67, 48,  8, 39, 37, 31, 34, 63, 79, 61, 47, 47, 64, 23,
       47, 26, 15, 19, 28, 31,  5, 41, 73, 31, 86, 73, 67, 54, 86, 67, 15,
       26, 29, 52, 19, 17, 13, 31, 12, 28, 81, 52, 61, 70,  1, 15, 62, 37,
       86,  2, 48, 61, 45, 59, 86,  0, 37, 25, 63, 19,  6, 79, 17, 19, 32,
       40, 36,  9, 60, 20, 84, 59, 53, 47, 81, 83, 18, 21, 13, 54, 69, 74,
       42, 21, 41, 56, 38, 28, 21, 17, 20,  3, 23, 23, 78, 10, 66,  1, 39,
       83, 42, 59, 32,  0, 18, 54,  2, 71, 42, 31, 32, 74,  4, 52, 71,  2,
       38, 66, 62, 45, 22, 53, 50, 73, 69, 50, 23, 84, 38, 46, 36,  0, 79,
       32,  8,  4, 60, 84

In [26]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "ause they told you to, why you listen to 'em?\nHands up, middle finger to 'em\nFuck that, get down\n\n[Hook]\nWhen I get down on my luck I hide behind my eyes in Hollywood\nThey say ain't what you know but who you know\nYou need to know someone to know no one\nWhen I get down on my luck\nI roll one up and roll around all by my lonesome\nLost some years, I us"

Next Char Predictions: 
 '9E(A[MO8BDrfHvbXhHG<-HtcbduhqoxzMG1exnZ8x\nGS%eM"*<x\ncin77y3>dyk.xoRbW\namq\']3S65CWuT pIZSgP\'GE>BcsaOOd6O9.2;>$Im>zmgVzg.9<T20,>+;uTaj .bEz!PaM[z\nE8c2%s02?HD(]3x[UOuw14,VinJ4IXF;403"66r)f GwJ[?\n1V!kJ>?n#Tk!FfbM5URmiR6xFND\ns?\'#]xV2LZ:NEn9w$AS0CsaLkw\no.zhx#bLZ3FY)VW3gR:F0DHW$5<SEGI,8s<<WAXR/l9ny/6/JDQ4Lh*g<xNDb(qqh7;n[--vHvq%t-oK$=<U<I%xDZR;f\'q-#Gi#p&W'


In [27]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())


Prediction shape:  (64, 350, 87)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.4658957


In [28]:
model.compile(optimizer='adam', loss=loss)

In [29]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [30]:
EPOCHS=10

In [31]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 182 steps
Epoch 1/10
182/182 [==============================] - 677s 4s/step - loss: 2.6715
Epoch 2/10
182/182 [==============================] - 676s 4s/step - loss: 1.8750
Epoch 3/10
182/182 [==============================] - 674s 4s/step - loss: 1.6219
Epoch 4/10
182/182 [==============================] - 668s 4s/step - loss: 1.4873
Epoch 5/10
182/182 [==============================] - 700s 4s/step - loss: 1.4017
Epoch 6/10
182/182 [==============================] - 674s 4s/step - loss: 1.3397
Epoch 7/10
182/182 [==============================] - 677s 4s/step - loss: 1.2890
Epoch 8/10
182/182 [==============================] - 671s 4s/step - loss: 1.2454
Epoch 9/10
182/182 [==============================] - 671s 4s/step - loss: 1.2059
Epoch 10/10
182/182 [==============================] - 671s 4s/step - loss: 1.1688


In [32]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [33]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            22272     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 87)             89175     
Total params: 4,049,751
Trainable params: 4,049,751
Non-trainable params: 0
_________________________________________________________________


In [35]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [38]:
print(generate_text(model, start_string=u"Text1: "))

Text1: Weven be such a stripper
Foreign, now that nigga ain't even gave
Make you got that weight and I said that it's fuckin' back to me, ho
This girl am I badgett a dollarry I'm a livin' in Dod (Uh), yeah, got my life, I
I might need to get feelin' me
Fucked up like a kison, worly wit' that thing gow

[Chorus 1: Villered Sends clappin'
No such thing ever done, feel the rain
'Cause I-aint done and don't do anything that you walk up with it
Man I bet you felt this faggot a city then I'm soardin'
Send the ground, put 'em in with those
No more big-and like a verse
Full of girl got the number and what
They started really deepun it when I'm gone, they bustin' at my chillage
Beating depending white Busen started than blams
In dollars of me, one fun
And when she don't got bars like that
What them lie like a movie nigga
That's feelin' forward when I'm from pay his nigga
Let it back on tom of the
Im just tryna view this much way before this sauce
Tell him, "Don't think that you thrown!"
The fac